In [ ]:
import random
from collections import Counter

import numpy as np

from preprocessing import get_tf_idf_twenty_newsgroup_corpus
from fuzzy_artmap_module import FuzzyArtMap
from fuzzy_artmap_module import complement_encode

valid_vector = np.array([[1.0], [0.0]])
invalid_vector = np.array([[0.0], [1.0]])

In [ ]:
corpus, categories, vectorizer = get_tf_idf_twenty_newsgroup_corpus()

In [ ]:
seed_indexes = [4000, 4001]
atheism_and_graphics = {index: category for index, category in categories.items() if (category == "alt.atheism" or category == "comp.graphics") and index not in seed_indexes}

In [ ]:
shuffled_document_indexes = random.sample(list(atheism_and_graphics.keys()), len(atheism_and_graphics))

In [ ]:
def get_test_input_and_output(doc_index, vector, relevant_category):
    if categories[doc_index] == relevant_category:
        output_value = valid_vector
    else:
        output_value = invalid_vector
    
    complement_encoded_input = complement_encode(vector.toarray().transpose())
    return complement_encoded_input, output_value

In [ ]:
fuzzy_artmap = FuzzyArtMap(corpus.shape[1]*2, 1, rho_a_bar=0.95)

In [ ]:
relevant_category = "alt.atheism"

In [ ]:
for iteration_count, document_index in enumerate(shuffled_document_indexes[:100]):
    print(iteration_count)
    input_vector, class_vector = get_test_input_and_output(document_index, corpus[document_index], relevant_category)
    fuzzy_artmap.train(input_vector, class_vector)

In [ ]:
accuracy_counter = Counter({"TP": 0, "TN": 0, "FP": 0, "FN": 0})
# predictions = []
for document_index in shuffled_document_indexes[100:500]:
    input_vector, class_vector = get_test_input_and_output(document_index, corpus[document_index], relevant_category)
    prediction, membership_degree = fuzzy_artmap.predict(input_vector)
    # if prediction[0][0]:
    #     predictions.append((membership_degree, document_index, class_vector[0][0]))
    # if prediction[0][0]:
    #     print(f"predicted: {prediction[0][0]} actual: {class_vector[0][0]} membership: {membership_degree} doc: {document_index}")
    # print(f"predicted: {prediction[0][0]} actual: {class_vector[0][0]} membership: {membership_degree}")
    if class_vector[0][0]:
        if prediction[0][0]:
            update = {"TP": 1}
        else:
            update = {"FN": 1}
    else:
        if prediction[0][0]:
            update = {"FP": 1}
        else:
            update = {"TN": 1}
    accuracy_counter.update(update)
print(accuracy_counter)
# predictions.sort(key=lambda p: p[0])
# print(predictions)

In [ ]:
def sample_document(vectorizer, corpus, document_index):
    features = vectorizer.get_feature_names_out()
    for feature_index in corpus[document_index].indices:
        print(features[feature_index])

In [ ]:
sample_document(vectorizer, corpus, 13938)